In [136]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('Scripts')

from SignalDataset import SignalDataset
from Dataloader import Dataloader
from SSVEPDataset import SSVEPSignalDataset
from SSVEPDataloader import SSVEPDataloaders
from utils import generate_time_series

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from TrainingFunc import train, test

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
# %run "SignalDatasetCreator.py"

In [41]:
dataset = SignalDataset()
dataloaders, dataset_sizes = Dataloader(dataset, batch_size = 128)

In [184]:
dataType = '512'
data = pd.read_csv('SSVEPDatasetPCA_'+dataType+'.csv')
ssvep_dataset = SSVEPSignalDataset(data)
ssvep_dataloaders, ssvep_datasets_izes = Dataloader(ssvep_dataset, batch_size = 16)

In [155]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.Linear1 = nn.Linear(512, 300)
        self.Linear2 = nn.Linear(300, 100)
        self.Linear3 = nn.Linear(100, 11)
        
    def forward(self, X):
        X = self.Linear1(X)
        X = self.Linear2(X)
        X = self.Linear3(X)
        return X

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
model = Model().double().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)
criterion = nn.CrossEntropyLoss()

In [158]:
model, training_stats, validation_stats = train(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, device,
                                                num_epochs = 2)



Epoch 0/1
----------
train Loss: 0.0001 Acc: 1.0000
val Loss: 0.0001 Acc: 1.0000

Epoch 1/1
----------
train Loss: 0.0001 Acc: 1.0000
val Loss: 0.0001 Acc: 1.0000

Training complete in 0m 1s
Best val Acc: 1.000000


In [124]:
test(model, dataloaders['test'], dataset_sizes['test'], criterion, device)

path = 'C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data\\Models\\SignalSelector'
torch.save(model.state_dict(), path+'\\selector.pth')

Test Loss: 0.0000 Acc: 1.0000


In [185]:
path = 'C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data\\ProcessedSignals\\512'
ssvep_dataloaders, _ = Dataloader(ssvep_dataset, batch_size = 1)
alpha = 10
i = [-1, -1, -1, -1, -1]
data = pd.DataFrame(columns = ['subject', 'state', 'path'])
for t in ['train', 'val', 'test']:
    dataloader = ssvep_dataloaders[t]
    for sample in dataloader:
        c = sample['class'].item()
        i[c] += 1
        state = c + 1
        new_path = path+'\\'+str(state)

        signal = sample['signal'].squeeze(0)

        _, frequency = torch.max(model(signal.to(device)), 1)
        frequency = frequency.item()

        signal = signal.numpy() + generate_time_series(frequency = frequency)

        new_path = new_path+'\\'+str(i[c])+'.csv'
        np.savetxt(new_path, signal)
        
        temp = pd.DataFrame([['_', state, new_path]] , columns = ['subject', 'state', 'path'])
        data = pd.concat([data, temp], axis = 0)
        
data.to_csv('SSVEPDatasetProcessed_512.csv', index = False)